In [29]:
#Step2: Data cleaning

In [117]:
##import your database
import pandas as pd
import numpy as np
df = pd.read_csv(r'xxxxxxxx', low_memory=False) #Import your csv data with your 
                 

In [119]:
##check data missing piece
print(df.info())
print(df.describe())
print(df.isnull().sum())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 513865 entries, 0 to 513864
Data columns (total 9 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   unix         513865 non-null  int64  
 1   date         513865 non-null  object 
 2   symbol       513865 non-null  object 
 3   open         513865 non-null  int64  
 4   high         513865 non-null  int64  
 5   low          513865 non-null  int64  
 6   close        513865 non-null  int64  
 7   Volume BTC   513865 non-null  float64
 8   Volume USDT  513865 non-null  float64
dtypes: float64(2), int64(5), object(2)
memory usage: 35.3+ MB
None
               unix           open           high            low  \
count  5.138650e+05  513865.000000  513865.000000  513865.000000   
mean   1.719885e+09   65878.769190   65882.466113   65874.604669   
std    9.088723e+06   14601.212215   14602.423597   14599.965944   
min    1.704067e+09   38550.000000   38555.000000   38533.000000   
25%    1.

In [121]:
##Data type exchange
df.columns = df.columns.str.strip()
df['date'] = pd.to_datetime(df['date'])
numeric_cols = ['open', 'high', 'low', 'close', 'Volume BTC', 'Volume USDT']
df[numeric_cols] = df[numeric_cols].astype(float)


In [123]:
##Repeating Data Examine
print(f"Repeating Line: {df.duplicated().sum()}")
df = df.drop_duplicates()
df = df.reset_index(drop=True)

Repeating Line: 0


In [125]:
##Data Consistency Examine
invalid_high_low = df[df['high'] < df['low']]
if not invalid_high_low.empty:
    print(f"Line {len(invalid_high_low)} high < low")
    df.loc[df['high'] < df['low'], ['high', 'low']] = df.loc[df['high'] < df['low'], ['low', 'high']].values
    
invalid_close = df[(df['close'] < df['low']) | (df['close'] > df['high'])]
if not invalid_close.empty:
    print(f"Line {len(invalid_close)} close is not between high-low range")
    df['close'] = np.where(df['close'] < df['low'], df['low'], 
                          np.where(df['close'] > df['high'], df['high'], df['close']))

In [127]:
##Adding Indicator
df['price_change'] = df['close'] - df['open']
df['price_range'] = df['high'] - df['low']
df['vwap'] = np.where(df['Volume BTC'] > 0, 
            df['Volume USDT'] / df['Volume BTC'], 
            df['close'])  

In [ ]:
print("\nCleaned Data Information:")
print(df.info())
print("\nCleaned Data Summary:")
print(df.describe())
df.to_csv('cleaned_crypto_data.csv', index=True)


Cleaned Data Information:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 513865 entries, 0 to 513864
Data columns (total 12 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   unix          513865 non-null  int64         
 1   date          513865 non-null  datetime64[ns]
 2   symbol        513865 non-null  object        
 3   open          513865 non-null  float64       
 4   high          513865 non-null  float64       
 5   low           513865 non-null  float64       
 6   close         513865 non-null  float64       
 7   Volume BTC    513865 non-null  float64       
 8   Volume USDT   513865 non-null  float64       
 9   price_change  513865 non-null  float64       
 10  price_range   513865 non-null  float64       
 11  vwap          513865 non-null  float64       
dtypes: datetime64[ns](1), float64(9), int64(1), object(1)
memory usage: 47.0+ MB
None

Cleaned Data Summary:
               unix                   